Phase 1: Business Understanding (The Foundation)**

Before we download the data, let's formally state the first phase we have just completed. This ensures our technical work is always guided by a clear business purpose.

*   **Objective:** To help an e-commerce business increase its profitability and customer retention.
*   **Business Problem:** The marketing and sales teams currently lack the insights needed for data-driven decision-making. They don't have a clear view of top-selling products, high-value customers, or cross-selling opportunities.
*   **Project Goal:** To build a centralized dashboard and recommendation system that provides actionable insights for sales analysis, customer segmentation, and product promotions. This will be our "single source of truth."
*   **Success Criteria:** The project is successful if we deliver a live dashboard, a working recommendation function, and a well-documented GitHub repository.

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
uploaded=files.upload()

Saving kaggle.json to kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print("\nKaggle API configured successfully!")


Kaggle API configured successfully!


In [4]:
!kaggle datasets list


ref                                                        title                                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  -------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
mosapabdelghany/medical-insurance-cost-dataset             Medical Insurance Cost Dataset                          16425  2025-08-24 11:54:36.533000          15145        298  1.0              
zadafiyabhrami/global-crocodile-species-dataset            Global Crocodile Species Dataset                        57473  2025-08-26 08:46:11.950000           9990        282  1.0              
codebynadiia/gdp-per-country-20202025                      GDP per Country 2020–2025                                5677  2025-09-04 14:37:43.563000           8204        156  1.0              
saadaliyaseen/analyzing-studen

In [5]:
!kaggle datasets download -d olistbr/brazilian-ecommerce

Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
License(s): CC-BY-NC-SA-4.0
  0% 0.00/42.6M [00:00<?, ?B/s]
100% 42.6M/42.6M [00:00<00:00, 474MB/s]


In [6]:
!unzip brazilian-ecommerce.zip

Archive:  brazilian-ecommerce.zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


In [7]:
print("\nDataset files have been downloaded and unzipped:")
!ls *.csv


Dataset files have been downloaded and unzipped:
olist_customers_dataset.csv	  olist_orders_dataset.csv
olist_geolocation_dataset.csv	  olist_products_dataset.csv
olist_order_items_dataset.csv	  olist_sellers_dataset.csv
olist_order_payments_dataset.csv  product_category_name_translation.csv
olist_order_reviews_dataset.csv


Phase 2: Data Understanding**

The goal of this phase is to become familiar with the data. We're not cleaning or changing anything yet; we're just exploring.

Our investigation will focus on three questions:
1.  What data is in each file? (Column names, data types)
2.  Are there any obvious quality issues? (Missing values, incorrect formats)
3.  How do the files relate to each other?

In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [9]:
data={}
for file in os.listdir("."):
  if file.endswith(".csv"):
    column_names=file.replace("olist_", "").replace("_dataset","").replace(".csv","")
    data[column_names]=pd.read_csv(file)
    print(f"Loaded '{file}' as data['{column_names}']")

Loaded 'olist_order_payments_dataset.csv' as data['order_payments']
Loaded 'olist_sellers_dataset.csv' as data['sellers']
Loaded 'product_category_name_translation.csv' as data['product_category_name_translation']
Loaded 'olist_order_reviews_dataset.csv' as data['order_reviews']
Loaded 'olist_customers_dataset.csv' as data['customers']
Loaded 'olist_products_dataset.csv' as data['products']
Loaded 'olist_geolocation_dataset.csv' as data['geolocation']
Loaded 'olist_orders_dataset.csv' as data['orders']
Loaded 'olist_order_items_dataset.csv' as data['order_items']


In [10]:
# We have customers information, orders information, products and order items information
# 1. Customers: Information about the customer
print("\n--- 1. Customers ---")
print(data['customers'].info())
print("\nMissing values in customers:")
print(data['customers'].isnull().sum())
print("\nSample customer data:")
print(data['customers'].head())


--- 1. Customers ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None

Missing values in customers:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

Sample customer data:
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2 

In [11]:
# 2. Orders: Connects customers to their purchases
print("\n\n--- 2. Orders ---")
print(data['orders'].info())
print("\nMissing values in orders:")
print(data['orders'].isnull().sum())
print("\nSample order data:")
print(data['orders'].head())



--- 2. Orders ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None

Missing values in orders:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carr

In [12]:
# 3. Order Items: Details of what was in each order (products, prices)
print("\n\n--- 3. Order Items ---")
print(data['order_items'].info())
print("\nMissing values in order_items:")
print(data['order_items'].isnull().sum())
print("\nSample order items data:")
print(data['order_items'].head())



--- 3. Order Items ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB
None

Missing values in order_items:
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

Sample order items data:
                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214    

In [13]:
# 4. Products: Details about the products themselves
print("\n\n--- 4. Products ---")
print(data['products'].info())
print("\nMissing values in products:")
print(data['products'].isnull().sum())
print("\nSample products data:")
print(data['products'].head())



--- 4. Products ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB
None

Missing values in products:
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty   

Phase 3: Data Preparation**

This is the Extract, Transform, Load (ETL) phase. I want to experience the practical steps followed in company so I am gonna save the data from its current messy state in CSV files into a clean, structured, and permanent home in a cloud database. A relational database like PostgreSQL is the ideal destination because it's built to handle the kind of multi-table relationships we see here. It has both free and paid. well, for practice purpose I will go with the free one.

In [14]:
# If you see here, I have installed psycopg2-binary. Why I did that? Basically it connects python
# to a PostgreSQL and it will allow me to run sql queries direclty from python. It means it allows
# python to communicate with the database. You will find how it works in further steps.
!pip install psycopg2-binary SQLAlchemy
print("Installed required libraries")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.0 MB/s eta 0:00:00
Installed required libraries


In [15]:
# The thing is that it is difficult to mention why I use all these libraries.
# I will do one thing. I will put everything in the readme file on github okay. you can check there.
# Don't blindly copy paste the code blocks. Just go through each block to understand what we did.
from sqlalchemy import create_engine, inspect
import glob
from google.colab import userdata
import time
print("Modules imported.")

Modules imported.


In [16]:
try:
    db_connection_str_raw = userdata.get('DB_CONNECTION_STRING')
    print("Successfully retrieved raw database connection string.")

    # --- DEBUGGING AND SANITIZING ---
    # 1. Strip any leading/trailing whitespace
    db_connection_str = db_connection_str_raw.strip()

    # 2. Ensure it starts with the correct prefix
    if db_connection_str.startswith("postgres://"):
        db_connection_str = db_connection_str.replace("postgres://", "postgresql://", 1)

    print("Sanitized connection string. Ready to create engine.")

except Exception as e:
    print(f"Error retrieving secret. Please ensure you have set 'DB_CONNECTION_STRING' in Colab secrets. Error: {e}")
    raise
print("Connected successfully")

Successfully retrieved raw database connection string.
Sanitized connection string. Ready to create engine.
Connected successfully


In [17]:
try:
    engine = create_engine(db_connection_str)
    # Test the connection immediately
    with engine.connect() as connection:
        print("✅ Database engine created and connection successful!")
except Exception as e:
    print(f"❌ Error creating engine or connecting to the database: {e}")
    print("\n--- TROUBLESHOOTING ---")
    print("1. Double-check your DB_CONNECTION_STRING in Colab Secrets. Make sure there are no typos.")
    print("2. Verify that your Neon database is active and not suspended.")
    print("3. Check that your password does not contain special characters that need URL encoding (e.g., @, :, /). If it does, you may need to generate a new password in Neon.")
    raise

✅ Database engine created and connection successful!


In [18]:
table_name_map = {
    'olist_customers_dataset.csv': 'customers',
    'olist_geolocation_dataset.csv': 'geolocation',
    'olist_order_items_dataset.csv': 'order_items',
    'olist_order_payments_dataset.csv': 'order_payments',
    'olist_order_reviews_dataset.csv': 'order_reviews',
    'olist_orders_dataset.csv': 'orders',
    'olist_products_dataset.csv': 'products',
    'olist_sellers_dataset.csv': 'sellers',
    'product_category_name_translation.csv': 'product_category_translation'
}

print("Schema definition complete.")

# --- 3. EXECUTE ETL: Loop through files and load to database ---

print("\n--- Starting the main ETL Process ---")
start_time = time.time()

# Get a list of all CSV files in the current Colab directory
csv_files = glob.glob('*.csv')
print(csv_files)

Schema definition complete.

--- Starting the main ETL Process ---
['olist_order_payments_dataset.csv', 'olist_sellers_dataset.csv', 'product_category_name_translation.csv', 'olist_order_reviews_dataset.csv', 'olist_customers_dataset.csv', 'olist_products_dataset.csv', 'olist_geolocation_dataset.csv', 'olist_orders_dataset.csv', 'olist_order_items_dataset.csv']


In [19]:
for file_path in csv_files:
    file_name = os.path.basename(file_path) # e.g., 'olist_customers_dataset.csv'

    # Process only the files we've defined in our schema map
    if file_name in table_name_map:
        table_name = table_name_map[file_name]
        print(f"\nProcessing file: '{file_name}'...")

        # -----------------
        # E - EXTRACT
        # -----------------
        # Read the raw data from the CSV file into a pandas DataFrame
        df = pd.read_csv(file_path)
        print(f"   (1/3) Extracted {len(df)} rows.")

        # -----------------
        # T - TRANSFORM
        # -----------------
        # The primary transformation is correcting the data types of date columns.
        # This is a critical data cleaning step for this specific dataset.
        for col in df.columns:
            if '_date' in col or '_timestamp' in col:
                # This line converts a text column to a proper datetime object.
                # If a value cannot be converted, it becomes 'NaT' (Not a Time).
                df[col] = pd.to_datetime(df[col], errors='coerce')
        print("   (2/3) Transformed data types for date columns.")


        # -----------------
        # L - LOAD
        # -----------------
        # This function writes the cleaned DataFrame to a table in your PostgreSQL database.
        # `if_exists='replace'` will drop the table if it already exists and create a new one.
        # `index=False` prevents pandas from adding an unnecessary index column.
        try:
            df.to_sql(table_name, engine, if_exists='replace', index=False)
            print(f"   (3/3) Loaded data into the '{table_name}' table in the database.")
        except Exception as e:
            print(f"❌ Error loading data for '{file_name}' into table '{table_name}': {e}")
            # If an error occurs during to_sql, attempt to rollback the transaction
            # This is important to reset the connection state for the next iteration
            try:
                with engine.connect() as connection:
                    connection.rollback()
                print(f"   Rolled back transaction for '{table_name}'.")
            except Exception as rollback_e:
                 print(f"   Error during rollback for '{table_name}': {rollback_e}")


# --- 4. VERIFICATION ---

end_time = time.time()
print(f"\n--- ETL Process Finished. Total time: {end_time - start_time:.2f} seconds ---")


Processing file: 'olist_order_payments_dataset.csv'...
   (1/3) Extracted 103886 rows.
   (2/3) Transformed data types for date columns.
   (3/3) Loaded data into the 'order_payments' table in the database.

Processing file: 'olist_sellers_dataset.csv'...
   (1/3) Extracted 3095 rows.
   (2/3) Transformed data types for date columns.
   (3/3) Loaded data into the 'sellers' table in the database.

Processing file: 'product_category_name_translation.csv'...
   (1/3) Extracted 71 rows.
   (2/3) Transformed data types for date columns.
   (3/3) Loaded data into the 'product_category_translation' table in the database.

Processing file: 'olist_order_reviews_dataset.csv'...
   (1/3) Extracted 99224 rows.
   (2/3) Transformed data types for date columns.
   (3/3) Loaded data into the 'order_reviews' table in the database.

Processing file: 'olist_customers_dataset.csv'...
   (1/3) Extracted 99441 rows.
   (2/3) Transformed data types for date columns.
   (3/3) Loaded data into the 'customers

In [20]:
print("\nStep 4: Verifying that all tables were created in the database...")
try:
    inspector = inspect(engine)
    tables_in_db = inspector.get_table_names()
    print("Verification successful. The following tables now exist:")
    for table_name in tables_in_db:
        print(f"  - {table_name}")
except Exception as e:
    print(f"❌ Could not verify tables. An error occurred: {e}")



Step 4: Verifying that all tables were created in the database...
Verification successful. The following tables now exist:
  - products
  - customers
  - geolocation
  - orders
  - order_items
  - product_recommendations
  - order_payments
  - sellers
  - product_category_translation
  - order_reviews


In [21]:
def run_sql(query, engine):
    """
    Executes a SQL query and returns the result as a pandas DataFrame.
    """
    print(f"Executing Query:\n---\n{query.strip()}\n---")
    return pd.read_sql_query(query, engine)

**Business Question 1: What are the top 10 best-selling products this month?**

In [22]:
from sqlalchemy import text

print("--- Running Diagnostic Checks on the 'orders' table ---")

# Diagnostic Query 1: Check the data type of the timestamp column in the DB
dtype_query = """
SELECT data_type
FROM information_schema.columns
WHERE table_name = 'orders' AND column_name = 'order_purchase_timestamp';
"""

# Diagnostic Query 2: Find the MIN and MAX dates to see the range
date_range_query = """
SELECT
    MIN(order_purchase_timestamp) as min_date,
    MAX(order_purchase_timestamp) as max_date,
    COUNT(*) as total_rows,
    COUNT(order_purchase_timestamp) as non_null_dates
FROM orders;
"""

try:
    # Execute and print results
    df_dtype = pd.read_sql(text(dtype_query), engine)
    print("\n1. Data Type of 'order_purchase_timestamp':")
    print(df_dtype)

    df_range = pd.read_sql(text(date_range_query), engine)
    print("\n2. Date Range and Null Count in 'order_purchase_timestamp':")
    print(df_range)

except Exception as e:
    print(f"An error occurred during diagnostics: {e}")

--- Running Diagnostic Checks on the 'orders' table ---

1. Data Type of 'order_purchase_timestamp':
                     data_type
0  timestamp without time zone

2. Date Range and Null Count in 'order_purchase_timestamp':
             min_date            max_date  total_rows  non_null_dates
0 2016-09-04 21:15:19 2018-10-17 17:30:18       99441           99441


In [23]:
!pip install psycopg2-binary SQLAlchemy
# --- Business Question 1 (Revised): Top 10 Best-Selling Products in the last 3 full months---
print("--- Query 1 (Revised): Top 10 Products (Last 3 Full Months) ---")

top_products_revised_query = """
SELECT
    COALESCE(pc.product_category_name_english, p.product_category_name, 'Unknown') AS product_category,
    COUNT(oi.order_id) AS units_sold
FROM
    order_items AS oi
JOIN
    orders AS o ON oi.order_id = o.order_id
JOIN
    products AS p ON oi.product_id = p.product_id
LEFT JOIN
    product_category_translation AS pc ON p.product_category_name = pc.product_category_name
WHERE
    o.order_purchase_timestamp >= (SELECT MAX(order_purchase_timestamp) FROM orders) - INTERVAL '3 months'
    AND o.order_status = 'delivered'
GROUP BY
    product_category
ORDER BY
    units_sold DESC
LIMIT 10;
"""

try:
    df_top_products = pd.read_sql(text(top_products_revised_query), engine)
    print(df_top_products)
except Exception as e:
    print(f"An error occurred: {e}")

--- Query 1 (Revised): Top 10 Products (Last 3 Full Months) ---
        product_category  units_sold
0          health_beauty        1203
1         bed_bath_table        1018
2             housewares         948
3         sports_leisure         734
4        furniture_decor         704
5          watches_gifts         681
6  computers_accessories         655
7                   auto         568
8              telephony         413
9              perfumery         341


In [24]:
# --- Business Question 2: Who are the top 5 highest-value customers? ---
print("\n--- Query 2: Top 5 Highest-Value Customers (Lifetime Spend) ---")

top_customers_query = """
SELECT
    c.customer_unique_id,
    SUM(op.payment_value) AS lifetime_value,
    COUNT(DISTINCT o.order_id) as total_orders
FROM
    customers c
JOIN
    orders o ON c.customer_id = o.customer_id
JOIN
    order_payments op ON o.order_id = op.order_id
WHERE
    o.order_status = 'delivered' -- Only count completed orders
GROUP BY
    c.customer_unique_id
ORDER BY
    lifetime_value DESC
LIMIT 5;
"""

try:
    df_top_customers = pd.read_sql(text(top_customers_query), engine)
    print(df_top_customers)
except Exception as e:
    print(f"An error occurred: {e}")


--- Query 2: Top 5 Highest-Value Customers (Lifetime Spend) ---
                 customer_unique_id  lifetime_value  total_orders
0  0a0a92112bd4c708ca5fde585afaa872        13664.08             1
1  da122df9eeddfedc1dc1f5349a1a690c         7571.63             2
2  763c8b1c9c68a0229c42c9fc6f662b93         7274.88             1
3  dc4802a71eae9be1dd28f5d788ceb526         6929.31             1
4  459bef486812aa25204be022145caa62         6922.21             1


In [25]:
!pip install psycopg2-binary SQLAlchemy

print("--- Query 3: Market Basket Analysis (Top 10 Product Pairs) ---")

market_basket_query = """
/*
 This query finds pairs of products that are frequently bought in the same order.
 STEP 1: (product_pairs) Self-join order_items to find all pairs of products
         within the same order. The condition 'a.product_id < b.product_id'
         is a trick to prevent duplicate pairs (e.g., [A,B] and [B,A]).
 STEP 2: Join with product and category tables to get human-readable names.
 STEP 3: Group by the category pair and count the frequency.
*/
WITH product_pairs AS (
    SELECT
        a.order_id,
        a.product_id AS product_a,
        b.product_id AS product_b
    FROM
        order_items a
    JOIN
        order_items b ON a.order_id = b.order_id AND a.product_id < b.product_id
)
SELECT
    COALESCE(pc1.product_category_name_english, 'Unknown') AS product_a_category,
    COALESCE(pc2.product_category_name_english, 'Unknown') AS product_b_category,
    COUNT(*) AS pair_frequency
FROM
    product_pairs pp
JOIN
    products p1 ON pp.product_a = p1.product_id
JOIN
    products p2 ON pp.product_b = p2.product_id
LEFT JOIN
    product_category_translation pc1 ON p1.product_category_name = pc1.product_category_name
LEFT JOIN
    product_category_translation pc2 ON p2.product_category_name = pc2.product_category_name
GROUP BY
    1, 2
ORDER BY
    pair_frequency DESC
LIMIT 10;
"""

try:
    df_market_basket = pd.read_sql(text(market_basket_query), engine)
    print(df_market_basket)
except Exception as e:
    print(f"An error occurred: {e}")

--- Query 3: Market Basket Analysis (Top 10 Product Pairs) ---
         product_a_category        product_b_category  pair_frequency
0            bed_bath_table            bed_bath_table            1147
1           furniture_decor           furniture_decor             555
2             health_beauty             health_beauty             347
3     computers_accessories     computers_accessories             325
4                housewares                housewares             306
5             watches_gifts             watches_gifts             215
6            sports_leisure            sports_leisure             189
7              garden_tools              garden_tools             179
8  fashion_bags_accessories  fashion_bags_accessories             154
9                      toys                      toys             123


In [26]:
!pip install psycopg2-binary SQLAlchemy

print("\n--- Query 4: Average Time Between 1st and 2nd Purchase ---")

time_between_purchases_query = """
/*
 This query calculates customer repurchase rate.
 STEP 1: (customer_order_rank) For each unique customer, rank their orders by date.
 STEP 2: (customer_pairs) Create pairs of consecutive orders (e.g., order 1 and 2,
         order 2 and 3, etc.) for each customer.
 STEP 3: (purchase_gaps) Calculate the time difference between these consecutive orders.
 STEP 4: Calculate the average time gap specifically for the first-to-second purchase.
*/
WITH customer_order_rank AS (
    SELECT
        c.customer_unique_id,
        o.order_id,
        o.order_purchase_timestamp,
        -- The ROW_NUMBER() window function assigns a rank to each order for a customer
        ROW_NUMBER() OVER(PARTITION BY c.customer_unique_id ORDER BY o.order_purchase_timestamp) as order_rank
    FROM
        orders o
    JOIN
        customers c ON o.customer_id = c.customer_id
    WHERE
        o.order_status = 'delivered'
),
first_second_purchase AS (
    SELECT
        t1.customer_unique_id,
        t1.order_purchase_timestamp AS first_purchase_date,
        t2.order_purchase_timestamp AS second_purchase_date,
        -- Calculate the difference between the two dates
        t2.order_purchase_timestamp - t1.order_purchase_timestamp AS time_to_repurchase
    FROM
        customer_order_rank t1
    JOIN
        customer_order_rank t2 ON t1.customer_unique_id = t2.customer_unique_id
    WHERE
        t1.order_rank = 1 AND t2.order_rank = 2
)
SELECT
    -- Calculate the final average and extract the number of days from the interval
    AVG(time_to_repurchase) AS average_repurchase_time
FROM
    first_second_purchase;
"""

try:
    df_time_between = pd.read_sql(text(time_between_purchases_query), engine)
    # The result might be a timedelta object, so let's format it nicely
    if not df_time_between.empty and 'average_repurchase_time' in df_time_between.columns:
         avg_time = df_time_between['average_repurchase_time'][0]
         print(f"The average time for a customer to make their second purchase is approximately: {avg_time.days} days.")
    else:
        print(df_time_between)

except Exception as e:
    print(f"An error occurred: {e}")



--- Query 4: Average Time Between 1st and 2nd Purchase ---
The average time for a customer to make their second purchase is approximately: 81 days.


In [27]:
!pip install psycopg2-binary SQLAlchemy
print("--- Query 3 (Improved): Cross-Category Market Basket Analysis ---")

market_basket_improved_query = """
WITH product_pairs AS (
    SELECT
        a.order_id,
        a.product_id AS product_a,
        b.product_id AS product_b
    FROM
        order_items a
    JOIN
        order_items b ON a.order_id = b.order_id AND a.product_id < b.product_id
),
category_pairs AS (
    SELECT
        pp.order_id,
        COALESCE(pc1.product_category_name_english, 'Unknown') AS product_a_category,
        COALESCE(pc2.product_category_name_english, 'Unknown') AS product_b_category
    FROM
        product_pairs pp
    JOIN
        products p1 ON pp.product_a = p1.product_id
    JOIN
        products p2 ON pp.product_b = p2.product_id
    LEFT JOIN
        product_category_translation pc1 ON p1.product_category_name = pc1.product_category_name
    LEFT JOIN
        product_category_translation pc2 ON p2.product_category_name = pc2.product_category_name
)
SELECT
    product_a_category,
    product_b_category,
    COUNT(*) AS pair_frequency
FROM
    category_pairs
WHERE
    -- THIS IS THE NEW CONDITION TO FIND CROSS-CATEGORY PAIRS
    product_a_category != product_b_category
GROUP BY
    1, 2
ORDER BY
    pair_frequency DESC
LIMIT 10;
"""

try:
    df_market_basket_improved = pd.read_sql(text(market_basket_improved_query), engine)
    print(df_market_basket_improved)
except Exception as e:
    print(f"An error occurred: {e}")

--- Query 3 (Improved): Cross-Category Market Basket Analysis ---
      product_a_category     product_b_category  pair_frequency
0         bed_bath_table        furniture_decor              71
1        furniture_decor         bed_bath_table              57
2           home_confort         bed_bath_table              52
3        furniture_decor           garden_tools              39
4             housewares         bed_bath_table              27
5                   baby                   toys              25
6             housewares        furniture_decor              25
7  computers_accessories                Unknown              24
8           garden_tools        furniture_decor              21
9            electronics  computers_accessories              19


In [28]:
!pip install -U scikit-learn -q
print("Installing/updating scikit-learn...")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 43.0 MB/s eta 0:00:00
Installing/updating scikit-learn...


In [29]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
print("libraries imported successfully.")

libraries imported successfully.


In [30]:
# --- 2. DATA EXTRACTION: Get interaction data from the database ---
# We need to know which customers bought which products.
print("\nFetching user-item interaction data from the database...")

interaction_query = """
SELECT
    c.customer_unique_id, -- Use the unique ID to track the person
    oi.product_id,
    COUNT(o.order_id) as purchase_count -- Could be used for weighting later
FROM
    orders o
JOIN
    customers c ON o.customer_id = c.customer_id
JOIN
    order_items oi ON o.order_id = oi.order_id
WHERE
    o.order_status = 'delivered'
GROUP BY
    1, 2;
"""
try:
    df_interactions = pd.read_sql(text(interaction_query), engine)
    print(f"Successfully fetched {len(df_interactions)} interaction records.")
except Exception as e:
    print(f"Error fetching data: {e}")
    raise

# Also, get a product lookup table for readable recommendations
products_query = """
SELECT
    p.product_id,
    COALESCE(pc.product_category_name_english, p.product_category_name, 'Unknown') AS category
FROM products p
LEFT JOIN product_category_translation pc
    ON p.product_category_name = pc.product_category_name;
"""
df_products = pd.read_sql(text(products_query), engine)
print(df_products)


Fetching user-item interaction data from the database...
Successfully fetched 99785 interaction records.
                             product_id                   category
0      1e9e8ef04dbcff4541ed26657ea517e5                  perfumery
1      3aa071139cb16b67ca9e5dea641aaa2f                        art
2      96bd76ec8810374ed1b65e291975717f             sports_leisure
3      cef67bcfe19066a932b7673e239eb23d                       baby
4      9dc1a7de274444849c219cff195d0b71                 housewares
...                                 ...                        ...
32946  a0b7d5a992ccda646f2d34e418fff5a0            furniture_decor
32947  bf4538d88321d0fd4412a93c974510e6  construction_tools_lights
32948  9a7c6041fa9592d9d9ef6cfe62a71f8c             bed_bath_table
32949  83808703fc0706a22e264b9d75f04a2e      computers_accessories
32950  106392145fca363410d287a815be6de4             bed_bath_table

[32951 rows x 2 columns]


In [31]:
# --- 2. DATA EXTRACTION: Get interaction data from the database ---
# This part remains the same.
print("\nFetching user-item interaction data from the database...")
interaction_query = """
SELECT c.customer_unique_id, oi.product_id, 1 as purchased
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY 1, 2;
"""
try:
    df_interactions = pd.read_sql(text(interaction_query), engine)
    print(f"Successfully fetched {len(df_interactions)} interaction records.")
except Exception as e:
    print(f"Error fetching data: {e}")
    raise
# Also, get a product lookup table
products_query = "SELECT product_id, COALESCE(pc.product_category_name_english, p.product_category_name, 'Unknown') AS category FROM products p LEFT JOIN product_category_translation pc ON p.product_category_name = pc.product_category_name;"
df_products = pd.read_sql(text(products_query), engine)


Fetching user-item interaction data from the database...
Successfully fetched 99785 interaction records.


In [32]:
# --- 3. EFFICIENT DATA PREPARATION FOR MODELING ---
print("\nPreparing data and building the sparse matrix directly (memory-efficient method)...")

# We need to convert our product and user IDs into unique integer codes.
df_interactions['product_id'] = df_interactions['product_id'].astype('category')
df_interactions['customer_unique_id'] = df_interactions['customer_unique_id'].astype('category')

product_codes = df_interactions['product_id'].cat.codes
customer_codes = df_interactions['customer_unique_id'].cat.codes
purchased_data = df_interactions['purchased'].values

# Create the sparse matrix directly from the codes
# Rows = product codes, Cols = customer codes, Data = purchased value (1)
product_user_matrix_sparse = csr_matrix((purchased_data, (product_codes, customer_codes)))

# Create lookup dictionaries to map back from codes to original IDs
# This is crucial for our recommendation function
code_to_product_id = {i: product for i, product in enumerate(df_interactions['product_id'].cat.categories)}
code_to_customer_id = {i: customer for i, customer in enumerate(df_interactions['customer_unique_id'].cat.categories)}

print("Sparse matrix built successfully. Shape:", product_user_matrix_sparse.shape)
# I have tried but, the thig is we are going to get like billions of elements inside it which cause
# memory issues. I am using google colab free version which only provides limited memory. So I took
# only limited data to check.



Preparing data and building the sparse matrix directly (memory-efficient method)...
Sparse matrix built successfully. Shape: (32216, 93358)


In [33]:
# --- 4. MODEL TRAINING ---
print("\nTraining the NearestNeighbors model...")
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(product_user_matrix_sparse)
print("Model training complete.")



Training the NearestNeighbors model...
Model training complete.


In [34]:
# --- 5. CREATE THE RECOMMENDATION FUNCTION (ADAPTED FOR CODES) ---

# We also need a map from the original product ID string to its new integer code
product_id_to_code = {product: i for i, product in enumerate(df_interactions['product_id'].cat.categories)}

def get_recommendations(product_id: str, model, matrix, n_recommendations: int = 5):
    """
    Takes a product_id string and returns n recommended products.
    """
    if product_id not in product_id_to_code:
        return f"Product ID '{product_id}' not found."

    # Convert our product_id string to its internal integer code
    product_code = product_id_to_code[product_id]

    distances, indices = model.kneighbors(matrix[product_code], n_neighbors=n_recommendations + 1)

    recommendations = []
    for i in range(1, len(distances.squeeze())):
        recommended_code = indices.squeeze()[i]
        recommended_product_id = code_to_product_id[recommended_code] # Map back to original ID
        similarity_score = 1 - distances.squeeze()[i]

        product_details = df_products[df_products['product_id'] == recommended_product_id]
        category = product_details['category'].iloc[0] if not product_details.empty else 'N/A'

        recommendations.append({
            'product_id': recommended_product_id,
            'category': category,
            'similarity': f"{similarity_score:.4f}"
        })
    return pd.DataFrame(recommendations)


In [35]:
# --- 6. EXAMPLE USAGE ---
most_popular_product_id = df_interactions['product_id'].value_counts().index[0]

print(f"\n--- Recommendations for a popular product (ID: {most_popular_product_id}) ---")
recommendations_df = get_recommendations(most_popular_product_id, model_knn, product_user_matrix_sparse)
print(recommendations_df)


--- Recommendations for a popular product (ID: 99a4788cb24856965c36a24e339b6058) ---
                         product_id         category similarity
0  35afc973633aaeb6b877ff57b2793310     home_confort     0.1098
1  f2e53dd1670f3c376518263b3f71424d   bed_bath_table     0.0531
2  b8be9a5f11908b7b70329f92fcb0eec9  furniture_decor     0.0468
3  b05fae603a3a28a977633c139cece058   bed_bath_table     0.0468
4  7aef20f12c90de905bf7782cb29dffb3  furniture_decor     0.0468


In [36]:
# ==============================================================================
# PHASE 5: EVALUATION - PRE-CALCULATING & STORING RECOMMENDATIONS (Corrected)
# ==============================================================================
# This version includes robust error handling for the database loading step
# to correctly manage transaction rollbacks.

import pandas as pd
print("Starting recommendation pre-calculation...")

# --- 1. Identify Top N Products ---
# (Assuming previous code block has run, otherwise re-run to define df_interactions etc.)
N = 1000
try:
    top_products = df_interactions['product_id'].value_counts().head(N).index.tolist()
    print(f"Identified the top {len(top_products)} most popular products.")
except NameError:
    print("Error: 'df_interactions' not found. Please re-run the previous model training cell.")
    raise

# --- 2. Calculate Recommendations for Each Top Product ---
all_recommendations = []
print(f"Calculating recommendations for each of the top {N} products... (This may take a minute)")

for i, product_id in enumerate(top_products):
    recs_df = get_recommendations(product_id, model_knn, product_user_matrix_sparse, n_recommendations=5)
    if isinstance(recs_df, pd.DataFrame) and not recs_df.empty:
        recs_df['source_product_id'] = product_id
        all_recommendations.append(recs_df)
    if (i + 1) % 100 == 0:
        print(f"   ...processed {i + 1} / {N} products.")

# --- 3. Compile and Format the Final DataFrame ---
if all_recommendations:
    final_recommendations_df = pd.concat(all_recommendations, ignore_index=True)
    final_recommendations_df = final_recommendations_df[['source_product_id', 'product_id', 'category', 'similarity']]
    final_recommendations_df.rename(columns={'product_id': 'recommended_product_id'}, inplace=True)
    print(f"\nSuccessfully generated a total of {len(final_recommendations_df)} recommendations.")
    print("Sample of the recommendations table to be loaded:")
    print(final_recommendations_df.head())

    # --- 4. Load Recommendations into the Database (Robust Method) ---
    print("\nLoading pre-calculated recommendations into the database...")

    # Establish a new connection to ensure a clean transaction state
    with engine.connect() as connection:
        try:
            # Begin a transaction
            with connection.begin():
                final_recommendations_df.to_sql(
                    'product_recommendations',
                    connection,  # Load using the connection, not the engine
                    if_exists='replace',
                    index=False
                )
            print("✅ Successfully created and populated the 'product_recommendations' table.")

        except Exception as e:
            print(f"❌ Error during database load: {e}")
            # The 'with connection.begin()' context manager automatically handles rollback on error.
            print("   Transaction has been rolled back.")

else:
    print("No recommendations were generated.")

# Let's verify the table exists after the operation
from sqlalchemy import inspect
inspector = inspect(engine)
if 'product_recommendations' in inspector.get_table_names():
    print("\nVerification successful: 'product_recommendations' table found in the database.")
else:
    print("\nVerification failed: 'product_recommendations' table was not created.")


Starting recommendation pre-calculation...
Identified the top 1000 most popular products.
Calculating recommendations for each of the top 1000 products... (This may take a minute)
   ...processed 100 / 1000 products.
   ...processed 200 / 1000 products.
   ...processed 300 / 1000 products.
   ...processed 400 / 1000 products.
   ...processed 500 / 1000 products.
   ...processed 600 / 1000 products.
   ...processed 700 / 1000 products.
   ...processed 800 / 1000 products.
   ...processed 900 / 1000 products.
   ...processed 1000 / 1000 products.

Successfully generated a total of 5000 recommendations.
Sample of the recommendations table to be loaded:
                  source_product_id            recommended_product_id  \
0  99a4788cb24856965c36a24e339b6058  35afc973633aaeb6b877ff57b2793310   
1  99a4788cb24856965c36a24e339b6058  f2e53dd1670f3c376518263b3f71424d   
2  99a4788cb24856965c36a24e339b6058  b8be9a5f11908b7b70329f92fcb0eec9   
3  99a4788cb24856965c36a24e339b6058  b05fae603a3a2

In [37]:
print("Installing necessary libraries...")
!pip install gspread gspread-dataframe google-auth-oauthlib -q


Installing necessary libraries...


In [38]:
from google.colab import drive
import gspread
from google.auth import default

# This is the key change: We use the 'drive' library which forces a robust auth flow.
# It also has the side benefit of mounting your Google Drive to the Colab environment.
print("\nAuthorizing this notebook to access your Google account...")
drive.mount('/content/drive')

# After mounting the drive, the credentials are in a state that gspread can use.
creds, _ = default()
gc = gspread.authorize(creds)

print("\n✅ Authentication and Drive mount successful.")
print("   You can now access your Google Drive files in the '/content/drive/My Drive/' folder.")


Authorizing this notebook to access your Google account...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

✅ Authentication and Drive mount successful.
   You can now access your Google Drive files in the '/content/drive/My Drive/' folder.


In [39]:
import pandas as pd
from sqlalchemy import text
from gspread_dataframe import set_with_dataframe

print("--- Starting Corrected Data Export ---")

# Connect to Google Sheets (assuming 'gc' is from the auth cell)
try:
    sh = gc.open('E-commerce Dashboard Data')
    print("Successfully opened the Google Sheet.")
except Exception as e:
    print(f"Could not open the Google Sheet. Please re-run the authentication cell. Error: {e}")
    raise

# Define the CORRECTED query with order_id
corrected_sales_query = """
SELECT
    o.order_id, -- <<< THE MISSING FIELD
    o.order_purchase_timestamp,
    op.payment_value,
    COALESCE(pc.product_category_name_english, p.product_category_name, 'Unknown') as product_category
FROM
    orders o
JOIN
    order_payments op ON o.order_id = op.order_id
JOIN
    order_items oi ON o.order_id = oi.order_id
JOIN
    products p ON oi.product_id = p.product_id
LEFT JOIN
    product_category_translation pc ON p.product_category_name = pc.product_category_name
WHERE
    o.order_status = 'delivered';
"""
print("Running corrected SQL query...")
df_sales_corrected = pd.read_sql(text(corrected_sales_query), engine)
print(f"Fetched {len(df_sales_corrected)} rows with the 'order_id' column.")

try:
    # Delete the old worksheet to replace it
    worksheet_to_delete = sh.worksheet('Sales_Overview')
    sh.del_worksheet(worksheet_to_delete)
    print("Deleted old 'Sales_Overview' tab.")
except gspread.exceptions.WorksheetNotFound:
    print("Old 'Sales_Overview' tab not found, will create a new one.")


# Create a new, clean worksheet and load the corrected data
ws_sales_new = sh.add_worksheet(rows=df_sales_corrected.shape[0]+1, cols=df_sales_corrected.shape[1], title="Sales_Overview")
set_with_dataframe(ws_sales_new, df_sales_corrected)

print("\n✅✅✅ SUCCESSFULLY FIXED: The 'Sales_Overview' tab in your Google Sheet now contains the correct data.")

--- Starting Corrected Data Export ---
Could not open the Google Sheet. Please re-run the authentication cell. Error: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x793d3ac8dd60>)


RefreshError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x793d3ac8dd60>)

In [ ]:
# I am so confused about this issuee. And I did everything manually like shifting the data to csv files
# and I built the dashboard using lookerstudio by google which free and beginner friendly also.
#I will share the link in github. You can check there and suggest me if there is any way to improve it.